#PySpark apply Function to Column


---

**How to apply a function to a column in PySpark? By using withColumn(), sql(), select() you can apply a built-in function or custom function to a column. In order to apply a custom function, first you need to create a function and register the function as a UDF. Recent versions of PySpark provide a way to use Pandas API hence, you can also use pyspark.pandas.DataFrame.apply().**

---

**Let’s create a PySpark DataFrame.**

In [0]:
columns = ["Seqno", "Name"]

data = [
    ("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")
]

df = spark.createDataFrame(data = data, schema=columns)
df.printSchema()
df.show(truncate=False)

root
 |-- Seqno: string (nullable = true)
 |-- Name: string (nullable = true)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+



##1. PySpark apply Function using withColumn()


---


**PySpark withColumn() is a transformation function that is used to apply a function to the column. The below example applies an upper() function to column df.Name.**

In [0]:
# Apply function using withColumn

from pyspark.sql.functions import upper

df.withColumn("Upper_Name", upper(df.Name))\
.show(truncate=False)

+-----+------------+------------+
|Seqno|Name        |Upper_Name  |
+-----+------------+------------+
|1    |john jones  |JOHN JONES  |
|2    |tracey smith|TRACEY SMITH|
|3    |amy sanders |AMY SANDERS |
+-----+------------+------------+



##2. Apply Function using select()


---


**The select() is used to select the columns from the PySpark DataFrame while selecting the columns you can also apply the function to a column.**

In [0]:
# Apply function using select

df.select("Seqno", "Name", upper(df.Name))\
.show(truncate=False)

+-----+------------+------------+
|Seqno|Name        |upper(Name) |
+-----+------------+------------+
|1    |john jones  |JOHN JONES  |
|2    |tracey smith|TRACEY SMITH|
|3    |amy sanders |AMY SANDERS |
+-----+------------+------------+



##3. Apply Function using sql()


---


**You can also apply the function to the column while running the SQL query on the PySpark DataFrame. In order to use SQL, make sure you create a temporary view using createOrReplaceTempView().**

**To run the SQL query use spark.sql() function and create the table by using createOrReplaceTempView(). This table would be available to use until you end your current SparkSession.**

**spark.sql() returns a DataFrame and here, I have used show() to display the contents to console.**

In [0]:
# Apply function using sql()

df.createOrReplaceTempView("TAB")

spark.sql(" select Seqno, Name, UPPER(Name) from TAB")\
.show(truncate=False)

+-----+------------+------------+
|Seqno|Name        |upper(Name) |
+-----+------------+------------+
|1    |john jones  |JOHN JONES  |
|2    |tracey smith|TRACEY SMITH|
|3    |amy sanders |AMY SANDERS |
+-----+------------+------------+



##4. PySpark apply Custom UDF Function


---


**In this section, I will explain how to create a custom PySpark UDF function and apply this function to a column.**

**PySpark UDF (a.k.a User Defined Function) is the most useful feature of Spark SQL & DataFrame that is used to extend the PySpark built-in capabilities. Note that UDFs are the most expensive operations hence use them only if you have no choice and when essential.**


---


**Following are the steps to apply a custom UDF function on an SQL query.**


###4.1 Create Custom Function

**First, create a python function. Though upper() is already available in the PySpark SQL function, to make the example simple, I would like to create one.**

In [0]:
# Create custom function
def upperCase(name_str):
    return name_str.upper()

###4.2 Register UDF


**Create a udf function by wrapping the above function with udf().**

In [0]:
#convert fuction to udf

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

upperCaseUDF = udf(lambda x: upperCase(x), StringType())

###4.3 Apply Custom UDF to Column


**Finally apply the function to the column by using withColumn(), select() and sql().**

In [0]:
# Custom udf with withColumn()

df.withColumn("Cureated Name", upperCaseUDF(col("Name")))\
.show(truncate=False)


# Custom function with select()

df.select(col("Seqno"),\
         upperCaseUDF(col("Name")).alias("Name"))\
.show(truncate=False)



# Custom function with sql()

spark.udf.register("upperCaseUDF", upperCaseUDF)
df.createOrReplaceTempView("TAB")

spark.sql(" select Seqno, Name, upperCaseUDF(Name) from TAB ")\
.show(truncate=False)





+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |JOHN JONES  |
|2    |TRACEY SMITH|
|3    |AMY SANDERS |
+-----+------------+

+-----+------------+------------------+
|Seqno|Name        |upperCaseUDF(Name)|
+-----+------------+------------------+
|1    |john jones  |JOHN JONES        |
|2    |tracey smith|TRACEY SMITH      |
|3    |amy sanders |AMY SANDERS       |
+-----+------------+------------------+



##5. PySpark Pandas apply()


**PySpark DataFrame doesn’t contain the apply() function however, we can leverage Pandas DataFrame.apply() by running Pandas API over PySpark. Below is a simple example to give you an idea.**

In [0]:
#Imports

import pyspark.pandas as ps
import numpy as np

technologies = ({
    'Fee' : [20000, 25000, 30000, 22000, np.NaN],
    'Discount' : [1000, 2500, 1500, 1200, 3000]
})


#Create DataFrame 

psdf = ps.DataFrame(data=technologies)
print(psdf)
print(f'\n\n')

def add(data):
    return data[0] + data[1]

addDF = psdf.apply(add, axis=1)
print(addDF)

       Fee  Discount
0  20000.0      1000
1  25000.0      2500
2  30000.0      1500
3  22000.0      1200
4      NaN      3000



0    21000.0
1    27500.0
2    31500.0
3    23200.0
4        NaN
dtype: float64
